# **Whole slide segmentation of Cell DIVE multiplex images using `DeepCell`**

## <span style="color: red"> !!NOTE!! THIS NOTEBOOK ACTS AS A TEMPLATE AND WILL NOT SAFE CHANGES UNLESS YOU SAVE A COPY TO YOUR OWN FOLDER. USE `FILE->DOWNLOAD` IN THE TASKBAR AT THE VERY TOP OF THIS PAGE TO SAVE A COPY OF THE NOTEBOOK. THEN, USING THE SIDEBAR TO THE RIGHT OPEN THE NEW COPY OF THE NOTEBOOK BEFORE YOU GET STARTED (RUNNING ON WSL C: AND D: DRIVES SHOULD BE LOCATED UNDER `drives/c` AND `drives/d`, RESPECTIVELY) !!NOTE!!</span> 

Run through this notebook **step-by-step** and adjust the code if necessary at any point in the process. The explaination and comments througout the notebook will help and guide through the proces. There will also be pointers as to what modification to a particular you might want to consider.

The structure of this notebook is adapted from and consequently loosely follows the structure and logic of the segmentation notebook of the `ark-analysis` toolset. Thus, it makes use some of the `ark-analysis` tooling as well

## Dependencies
We load all the necessary libraries used for the segmentation, visualisation and quantification in this notebook.

In [ ]:
# import required packages
import os
import warnings
from pathlib import Path
import pathlib
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io as io
import xarray as xr
from alpineer import io_utils, load_utils, image_utils

from ark.segmentation import marker_quantification, segmentation_utils
from ark.utils import (deepcell_service_utils, plot_utils)

from tensorflow.keras.models import load_model

from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import make_outline_overlay
from deepcell.utils.plot_utils import create_rgb_image

from wsi_segmentation.segmentation_utils import predict_tiled
from wsi_segmentation.io_utils import save_model_output_wrapper

from tqdm.notebook import tqdm
from ipyfilechooser import FileChooser

## 1. Define data location and setup directory structure

### 1.1 Set data directory

The first step is to set the *data/base directory* `base_dir` that contains a folder called `ome_tiff` with 1 *ome.tiff* per slide. The content of the *base* directory would look something like example below. We will also save the results and all the pipeline outputs in this folder as well.

```
└── ome_tiff
    ├── slide_asdfghjk.ome.tiff
    └── slide_qwertyui.ome.tiff
    
```

In order to define the *data/base directory* we have two options:
* set `set_base_dir_method="FilePicker"` and use the widget to find and identify the data/base directory
* set `set_base_dir_method="Manual"` and manual define the data/base directory below via `data_dir=`
* set `set_base_dir_method="Relative"` which assume that this notebook is saved in a `notebook` folder alongside the `ome_tiff` in the same parent directory

The last option `` ensure greatest reproducibility and transferability as the notebook is saved alongside the data and all path are defined relative to it.

**NOTE: In `WSL` the `C:` drive, `D:` drive, etc are mounted and located at `/mnt/c`, `/mnt/d`, etc, respectively.**

In [ ]:
set_base_dir_method = "Relative"

In [ ]:
if set_base_dir_method == "FilePicker":
    fc = FileChooser("")
    fc.show_only_dirs = True
    fc.title = '<b>Choose data folder that contains `ome_tiff` folder with 1 ome_tiff per slide</b>'
    display(fc)
elif set_base_dir_method == "Relative":
    notebook_dir = %pwd
    data_dir = os.path.dirname(notebook_dir)
elif set_base_dir_method == "Manual":
    data_dir = "/path/to/data/directory"

In [ ]:
# set the base directory
base_dir = fc.selected_path if set_base_dir_method == "FilePicker" else data_dir

### 1.2 Setup intermediate and results paths

Next, we create all the required output and results directories such that we have the following folder structure in the `base directory`:

```
├── ome_tiff
├── image_data
└── segmentation
    ├── cell_table
    ├── cell_table_no_qc
    ├── deepcell_input
    ├── deepcell_output
    ├── deepcell_output_no_qc
    ├── deepcell_visualization
    └── deepcell_visualization_no_qc
    
```

Here, we set all the directory names to be created.

In [ ]:
ome_tiff_dir = os.path.join(base_dir, "ome_tiff")
tiff_dir = os.path.join(base_dir, "image_data")
cell_table_dir = os.path.join(base_dir, "segmentation/cell_table_no_qc")
cell_table_dir_qc = os.path.join(base_dir, "segmentation/cell_table")
deepcell_input_dir = os.path.join(base_dir, "segmentation/deepcell_input")
deepcell_output_dir = os.path.join(base_dir, "segmentation/deepcell_output_no_qc")
deepcell_output_dir_qc = os.path.join(base_dir, "segmentation/deepcell_output")
deepcell_visualization_dir = os.path.join(base_dir, "segmentation/deepcell_visualization_no_qc")
deepcell_visualization_dir_qc = os.path.join(base_dir, "segmentation/deepcell_visualization")

Then, we check if the directories exist and if not create them.

In [ ]:
for directory in [tiff_dir, cell_table_dir, cell_table_dir_qc, deepcell_input_dir, deepcell_output_dir, deepcell_output_dir_qc, deepcell_visualization_dir, deepcell_visualization_dir_qc]:
    if not os.path.exists(directory):
        os.makedirs(directory)

Finally, validate that all the directories have been created correctly, so we can proceed with the rest of the notebook.

In [ ]:
# validate paths
io_utils.validate_paths([base_dir,
                         tiff_dir,
                         deepcell_input_dir,
                         deepcell_output_dir,
                         deepcell_output_dir_qc,
                         cell_table_dir,
                         cell_table_dir_qc,
                         deepcell_visualization_dir,
                         deepcell_visualization_dir_qc
])

## 2. Data preprocessing

### 2.1 Split OME TIFF files into single-channel TIFFs

Assuming we have obtained multichannel `ome.tiff` files from the Cell DIVE platform, we have to split them into single channel `tiff` files. To achieve this, set `split_ome = True` and split the `ome.tiff` by running the below steps. If you have run this step previously and already split up the `ome.tiff` files you can set `split_ome = False` to skip this step.

Alternatively, you can also manually create one folder per image/slide containing single channel `tiff`s inside the `image_data` folder in the `base/data folder`.

In [ ]:
split_ome = True

In [ ]:
if split_ome:
    # Gather all the OME-TIFFs
    ome_dir = pathlib.Path(ome_tiff_dir)
    ome_tiffs = list(ome_dir.glob("*.ome.[tif tiff]*"))

    os.listdir(ome_tiff_dir)

In [ ]:
# # Convert all the OME-TIFFs to Single Channel TIFFs
if split_ome:
    for ome_tiff in tqdm(ome_tiffs):
        load_utils.ome_to_fov(ome_tiff, data_dir=tiff_dir)

After these steps we should have one directory per slide in the `image_data` directory containing single `.tiff` files per channel. Looking similiar to this:

```
├── ome_tiff
│   ├── slide_asdfghjk.ome.tiff
│   └── slide_qwertyui.ome.tiff
└── image_data
    ├── slide_asdfghjk
    │   ├── ChannelNameA.tiff
    │   ├── ChannelNameB.tiff
    │   ├── ChannelNameC.tiff
    │   ├── ChannelNameD.tiff
    │   ├── ChannelNameE.tiff
    │   ├── ChannelNameF.tiff
    │   ├── ChannelNameG.tiff
    │   ├── ChannelNameH.tiff
    │   ├── ChannelNameI.tiff
    │   ├── ChannelNameJ.tiff
    │   ├── ChannelNameK.tiff
    │   ├── .
    │   ├── .
    │   └── .
    └── slide_qwertyui
        ├── ChannelNameA.tiff
        ├── ChannelNameB.tiff
        ├── ChannelNameC.tiff
        ├── ChannelNameD.tiff
        ├── ChannelNameE.tiff
        ├── ChannelNameF.tiff
        ├── ChannelNameG.tiff
        ├── ChannelNameH.tiff
        ├── ChannelNameI.tiff
        ├── ChannelNameJ.tiff
        ├── ChannelNameK.tiff
        ├── .
        ├── .
        └── .
        
```

### 2.2 Filter slides

We might not want to perform the segmentation and analysis on all the slides/images. If that is the case you can adjust and filter for the subset of slides you want to analyse below.

In [ ]:
# get all fovs in the folder...
wsis = io_utils.list_folders(tiff_dir)

# ... or optionally, select a specific set of fovs manually by name
# wsis = [*np.asarray(wsis)[np.isin(wsis, ["slide_name"])]]
print(wsis)

We can also list the channel for all the slide:

In [ ]:
for wsi in tqdm(wsis):
    print(wsi+" channels:")
    print(io_utils.remove_file_extensions(io_utils.list_files(os.path.join(tiff_dir, wsi))))

### 2.3 Perform hard thresholding for some of the channels

**!! NOTE: You could perform hard thresholding at this point by modifying the single channel images before creating composite channels for segmentation in the next step. !!**

In [ ]:
### you can manual implement hard thresholding of selected channel here and overwrite the respective single channel tiffs in the image_data subfolders with the thresholded image.

### 2.4 Load images and compute composite nuclear and membrane channels and save as `Mesmer` compatible input

Next, we define the the *nuclear* `nuc` and *membrane* `mems` channels that will be used as input for the segmentation. Multiple *membrane* or *nuclear* channels will be collapsed into a singular channel, respectively. This can be useful if there is not a singular marker that is ubiquitously expressed by all cells in the tissue in question. At least one of `nucs` and `mems` can not be `None`. If the channel/channel names are exactly the same across all slides you can simple define an array of channel names as done by default below. Alternatively, if you want to use a different set of channels for each slide or the channel names are not consistent you will need to define a `dictionary` with and entry for each slide (e.g. `mems = {"slide_asdfghjk": ['HLADR', 'CD68', 'CK8', 'CD3', 'CD45', 'VIM'], "slide_qwertyui": ['HLAA', 'HLADR', 'CD68', 'CK8', 'CD3', 'CD45', 'VIMENTIN']}`). Note the difference naming of the `VIMENTIN` channel between the images and that `slide_asdfghjk` is missing the `HLAA` channel.

In [ ]:
# nuclear channel name(s) (or nucs = None)
nucs = ['DAPI_FINAL']
# `nucs = {"slide_asdfghjk": ['DAPI_FINAL'],
# "slide_qwertyui": ['DAPI_FINAL']}

# # membrane channel name(s) (or mems = None)
mems = ['HLAA', 'HLADR', 'CD68', 'CK8', 'CD3', 'CD45', 'VIM']
# `mems = {"slide_asdfghjk": ['HLADR', 'CD68', 'CK8', 'CD3', 'CD45', 'VIM'],
# "slide_qwertyui": ['HLAA', 'HLADR', 'CD68', 'CK8', 'CD3', 'CD45', 'VIMENTIN']}

Once we have defined the `nuclear` and `membrane` markers, we create the composite nuclear-membrane `tiff` files which will used as the input to the `Deepcell` segmentation model.

In [ ]:
# generate and save deepcell input tiffs
# set img_sub_folder param to None if the image files in tiff_dir are not in a separate sub folder 
for wsi in tqdm(wsis):
    nucs_channels = nucs if isinstance(nucs, dict) or nucs == None else nucs[wsi]
    mems_channels = mems if isinstance(mems, dict) or mems == None else mems[wsi]

    deepcell_service_utils.generate_deepcell_input(
        deepcell_input_dir,
        tiff_dir,
        nucs_channels,
        mems_channels,
        [wsi],
        img_sub_folder=None
    )

Following this step, we have creates a single `tiff` file per slide inside the `segmentation/deepcell_input` folder. This should looks similar to this:

```
└── segmentation
    └──  deepcell_input
        ├── slide_asdfghjk.tiff
        └── slide_qwertyui.tiff
```

## 3. Whole Slide Segmentation via `Deepcell`'s `Mesmer` model

We use the the pre-trained `Mesmer` `MultiplexSegmentation` model provided by the `Deepcell` library. `Mesmer` was trained on 20X resolution data and thus depending on the our data we have to rescale the image by setting the `rescale_factor`, such that for 10X image `rescale_factor=2.0` or for a 60X image `rescale_factor=0.33`.

In [ ]:
rescale_factor = 1.0

### 3.1 Load `Mesmer` segmentation model

Let's load the pre-trained `Mesmer` model from the hardrive or redownload by setting `keras_model` to `keras_model = None`. 

**!! NOTE: Ignore NUMA node warnings. These warnings are docker/singularity container specific and can be safely ignored. !!**

In [ ]:
# set keras_model = None to redownload the latest MultiplexSegmentation
keras_model = load_model(Path("/.keras/models/MultiplexSegmentation"), compile=False)
segmentation_model = Mesmer() if keras_model == None else Mesmer(model=keras_model)

### 3.2 Run segmentation

A final check and chance to adjust and filter the slides on which we will perform the cell segmentation on. By default it will only run the segmentation on the slides selected in **Section 2.2**.

In [ ]:
slide_file_names = np.asarray(io_utils.list_files(os.path.join(deepcell_input_dir)))
slide_file_names = [*slide_file_names[np.isin(io_utils.remove_file_extensions(slide_file_names), wsis)]]

In [ ]:
slide_file_names

The segmentation function `predicted_tiled` takes various arguments that can/should be adapted for a given experiment:
* overlap: the amount of pixels that tiles of the whole slide overlap during the segmentation in case the slide is too large to be processed and segmented all at once. Choose larger overlaps if the cells imaged are large to reduce artifacts.
* cutoff: defines how close to a border of a segmentation tiles a cell has to be in order to be removed and inferred via `infer_gaps=True`
* background_threshold: the maximum expected background signal for an area of the image/slide that is not containing any cells
* infer_gaps: if set to `True` the boundary regions (see `cutoff`) of a segmentation tile are inferred from overlapping tiles (see `overlap`); set to `False` leaves boundaries between tiles unsegmented and the overlap setting will be ignored
* compartment: the cellular compartment that will be segmented for; can be either "whole-cell", "nuclear" or "both"
* cell_size_threshold: defines the minimum size of a segmented cells in pixels to be accepted as a "cell"; Default=None
* app: `keras` model used for segmentation
* pre_processing: pre-processing arguments passed on to the `deepcell` method (see https://deepcell.readthedocs.io/en/master/API/deepcell.applications.html#deepcell.applications.mesmer.Mesmer.predict)
* post_processing: post-processing arguments passed on to `deepcell` method (see https://deepcell.readthedocs.io/en/master/API/deepcell.applications.html#deepcell.applications.mesmer.Mesmer.predict)

At the moment this notebook only officially supports `whole-cell` segmentation. If you run `nuclear` or `both` segmentation you may need to adjust some parts in the QC section below.

In [ ]:
overlap = 200
cutoff = 2
background_threshold = 0.1
infer_gaps = True
compartment = "whole-cell"
cell_size_threshold = None ## we perform cell size thresholding during the QC step below 
app = segmentation_model

In [ ]:
for slide_file_name in tqdm(slide_file_names):
    slide = load_utils.load_imgs_from_dir(deepcell_input_dir, files = [slide_file_name], xr_channel_names=["nuclear", "membrane"])
    segmentation_mask = predict_tiled(slide.data, overlap=overlap, cutoff=cutoff, background_threshold=background_threshold, infer_gaps=infer_gaps, compartment=compartment, cell_size_threshold=cell_size_threshold, app=app)
    save_model_output_wrapper(segmentation_mask, output_dir=deepcell_output_dir, feature_name=io_utils.remove_file_extensions([slide_file_name])[0],compartment=compartment)

## 4. Visualisation

### 4.1 Visualise the segmented mask overlaid on the nuclear and membrane channels used for the segmentation 

To check the results of the segmentation, we can visualise the segmentation mask overlayed on the *nuclear* and *membrane* channel used for the segmentation. You can adjust the `id_to_visualise` variable below to plot result of a different slide.

In [ ]:
# create rgb overlay of image data for visualization
id_to_visualise = 0
segmentation_channels = load_utils.load_imgs_from_dir(deepcell_input_dir, files = [io_utils.list_files(os.path.join(deepcell_input_dir))[id_to_visualise]], xr_channel_names=["nuclear", "membran"])
segmentation_mask = load_utils.load_imgs_from_dir(deepcell_output_dir, files = [io_utils.list_files(os.path.join(deepcell_output_dir))[id_to_visualise]])
rgb_images = create_rgb_image(np.asarray(segmentation_channels[:,:,:,:]), channel_colors=['green', 'blue'])
overlay_segmentation = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_mask)

In [ ]:
# plot the data
fig, ax = plt.subplots(1 ,figsize=(15, 15))
ax.imshow(overlay_segmentation[0, ...])
ax.set_title('')

plt.show()

### 4.2 Save and visualise all segmentation outputs

Now, we save the segmentation mask overlay for each slide in the `segmentation/visualisation_no_qc` directory. You can also visualise a given channel on top of the segmentation by defining the `channel=` argument below with a list `[]` of channel names.

In [ ]:
# save the overlaid segmentation labels for each fov (these will not display, but will save in viz_dir)
segmentation_utils.save_segmentation_labels(
    segmentation_dir=deepcell_output_dir,
    data_dir=deepcell_input_dir,
    output_dir=deepcell_visualization_dir,
    fovs=io_utils.remove_file_extensions(wsis),
    channels=None
)

## 5. Quantification and Quality Control

### 5.1 Feature extraction and quantification

In this step we will perform feature extraction and quantification. For a full list of the features extracted, you can refer to the cell table section of at the `ark-analysis` [documentation](https://ark-analysis.readthedocs.io/en/latest/_rtd/data_types.html). Set the `nuclear_counts` and `fast_extraction` variables below to determine the level of information per cell that is being extracted and calculated. We generate both a size-normalised as well as a `arcsinh` transformed expression matrix for the segmented cells.

In [ ]:
# set to True to add nuclear cell properties to the expression matrix
nuclear_counts = False

# set to True to bypass expensive cell property calculations
# only cell label, size, and centroid will be extracted if True
fast_extraction = True

In [ ]:
# now extract the segmented imaging data to create normalized and transformed expression matrices
# note that if you're loading your own dataset, please make sure all the imaging data is in the same folder
# with each fov given its own folder and all fovs having the same channels
cell_table_size_normalized = dict()
cell_table_arcsinh_transformed = dict()

for wsi in tqdm(wsis):
    cell_table_size_normalized[wsi], cell_table_arcsinh_transformed[wsi] = \
        marker_quantification.generate_cell_table(segmentation_dir=deepcell_output_dir,
                                                tiff_dir=tiff_dir,
                                                img_sub_folder=None,
                                                fovs=[wsi],
                                                nuclear_counts=nuclear_counts,
                                                fast_extraction=fast_extraction)

We can now save the size-normalised and `arcsinh` transformed expression matrices as `csv` files to hardrive. The compression of these `csv` files can optionally be set via `compression = `

In [ ]:
# Set the compression level if desired, ZSTD compression can offer up to a 60-70% reduction in file size.
# NOTE: Compressed `csv` files cannot be opened in Excel. They must be uncompressed beforehand.
compression = None

# Uncomment the line below to allow for compressed `csv` files.
# compression = {"method": "zstd", "level": 3}
for wsi in tqdm(wsis):
    cell_table_size_normalized[wsi].to_csv(os.path.join(cell_table_dir, 'cell_table_size_normalized_'+ wsi + '.csv'),
                                    compression=compression, index=False)
    cell_table_arcsinh_transformed[wsi].to_csv(os.path.join(cell_table_dir, 'cell_table_arcsinh_transformed_'+ wsi + '.csv'),
                                        compression=compression, index=False)

### 5.2 Perform Cell DIVE specific QC (DAPI staining deviation and cell size)

Additionally, we perform basic quality control to improve the quality of the data and account for Cell DIVE specific issues such as the loss of cells over multiple rounds of staining (large deviation of DAPI staining between first staining round and final staining round).

In particular we perform two QC steps:
* remove any cell that is smaller than 50 pixel in area
* remove any cell for which the first and last DAPI staining shows a deviation > 0.5 

In [ ]:
cell_table_size_normalized_qc = dict()
for wsi in tqdm(wsis):
    cell_table_size_normalized_qc[wsi] = cell_table_size_normalized[wsi].loc[cell_table_size_normalized[wsi]["cell_size"] >= 50,]
    cell_table_size_normalized_qc[wsi] = cell_table_size_normalized_qc[wsi].loc[abs(cell_table_size_normalized_qc[wsi]["DAPI_INIT"] - cell_table_size_normalized_qc[wsi]["DAPI_FINAL"])/cell_table_size_normalized_qc[wsi]["DAPI_INIT"] <= 0.5, ]

In [ ]:
cell_table_arcsinh_transformed_qc = dict()

for wsi in tqdm(wsis):
    array_mask = np.isin(np.array(cell_table_arcsinh_transformed[wsi]["label"]), np.array(cell_table_size_normalized_qc[wsi]["label"]))
    df = pd.DataFrame.copy(cell_table_arcsinh_transformed[wsi])
    df = pd.DataFrame(df.loc[array_mask,])
    cell_table_arcsinh_transformed_qc[wsi] = df 


We perform a quick sanity check that both the normalised and transformed data after QC still contain the same cell IDs.

In [ ]:
for wsi in wsis:
    print(wsi + ":")
    print(np.array(cell_table_size_normalized_qc[wsi]["label"]).sort() == np.array(cell_table_arcsinh_transformed_qc[wsi]["label"]).sort())

We save the cleaned expression matrices as `csv` files to hardrive. The compression of these files can optionally be set via `compression=`.

In [ ]:
compression = None

for wsi in tqdm(wsis):
    cell_table_size_normalized_qc[wsi].to_csv(os.path.join(cell_table_dir_qc, 'cell_table_size_normalized_'+ wsi + '.csv'),
                                    compression=compression, index=False)
    cell_table_arcsinh_transformed_qc[wsi].to_csv(os.path.join(cell_table_dir_qc, 'cell_table_arcsinh_transformed_'+ wsi + '.csv'),
                                        compression=compression, index=False)

Lastly, we also apply QC results to the segmentation masks. We remove the filtered cells identified in the above steps from the segmentation mask:

In [ ]:
for wsi in tqdm(wsis):
    
    tmp_ids_qc = np.array(cell_table_size_normalized_qc[wsi]["label"])

    tmp_segmentation_mask = load_utils.load_imgs_from_dir(deepcell_output_dir, files = [wsi + "_whole_cell.tiff"])

    tmp_segmentation_mask = np.where(np.isin(tmp_segmentation_mask, tmp_ids_qc, invert=True), 0, tmp_segmentation_mask.data)

    save_model_output_wrapper(tmp_segmentation_mask, output_dir=deepcell_output_dir_qc, feature_name=wsi,compartment=compartment)

No, we save the updated segmentation mask overlay after QC for each slide in the `segmentation/visualisation` directory. You can also visualise a given channel on top of the segmentation by defining the `channel=` argument below with a list `[]` of channel names.

In [ ]:
# save the overlaid segmentation labels for each fov after QC (these will not display, but will save in viz_dir)
segmentation_utils.save_segmentation_labels(
    segmentation_dir=deepcell_output_dir_qc,
    data_dir=deepcell_input_dir,
    output_dir=deepcell_visualization_dir_qc,
    fovs=io_utils.remove_file_extensions(wsis),
    channels=None
)

At this point, we should have something that resembles the following folder structure and files. This folder/data structure should be readibly usable with the `ark-analysis` [pipeline](https://ark-analysis.readthedocs.io/en/latest/) notebooks starting from [Pixel clustering with Pixie](https://github.com/angelolab/ark-analysis#2-pixel-clustering-with-pixie) notebook.

```markdown
└── segmentation
    ├── cell_table
    │   ├── cell_table_arcsinh_transformed.csv
    │   └── cell_table_size_normalized.csv
    ├── cell_table_no_qc
    │   ├── cell_table_arcsinh_transformed.csv
    │   └── cell_table_size_normalized.csv
    ├── deepcell_input
    │   ├── slide_asdfghjk.tiff
    │   └── slide_qwertyui.tiff
    ├── deepcell_output
    │   ├── slide_asdfghjk_whole_cell.tiff
    │   └── slide_qwertyui_whole_cell.tiff
    ├── deepcell_output_no_qc
    │   ├── slide_asdfghjk_whole_cell.tiff
    │   └── slide_qwertyui_whole_cell.tiff
    ├── deepcell_visualization
    │   ├── slide_asdfghjk_segmentation_borders.tiff
    │   └── slide_qwertyui_segmentation_borders.tiff
    └── deepcell_visualization_no_qc
        ├── slide_asdfghjk_segmentation_borders.tiff
        └── slide_qwertyui_segmentation_borders.tiff
```

## 6. Create output that can be used in other pipelines

This section will provide code to transform/translate the above segmentation result into different formats and structure so they can be used with other existing analysis pipeline. You can also use it to define your own transformation/restructuring function. By default an output compatible with the `ark analysis` toolkit is being created.

In [ ]:
ark_output = True

In [ ]:
spooxs_output = False

In [ ]:
other_output = False

### 6.1 Ark analysis toolkit

The ark analysis pipeline assumes all extracted features to be stored in single dataframe and thus we have to harmonize the channel names if necessary and remove channels that do not appear in all slides. The default below assumes that all images have exactly the same channels and used the same naming scheme. You can modify the marker names by defining key-value pairs (e.g. `"old_channel_name":"new_channel_name"`) in the `channel_rename_dictionary` below. Not dealing with mismatched marker names or removing markers that are not present in all images may lead to issue further downstream in the analysis. If you want proceed with mismatched column names regardless you can set `ignore_na_warning=True`. By default, we will remove markers that do not appear in all slides, if you would like to keep all markers in the final quantification table, acknowledging that this will introduce `NA`s, set `remove_non_shared_markers=False`.

In [ ]:
## modify or adjust channel names if necessary
marker_rename_dictionary = {    
# "old_marker_name":"new_marker_name"
}

for wsi in wsis:
    cell_table_size_normalized[wsi].rename(columns=marker_rename_dictionary, inplace=True)
    cell_table_arcsinh_transformed[wsi].rename(columns=marker_rename_dictionary, inplace=True)
    cell_table_size_normalized_qc[wsi].rename(columns=marker_rename_dictionary, inplace=True)
    cell_table_arcsinh_transformed_qc[wsi].rename(columns=marker_rename_dictionary, inplace=True)

In [ ]:
ignore_na_warning = False
remove_non_shared_markers = True

In [ ]:
if ark_output:
    ## non qc
    combined_cell_table_size_normalized = pd.concat(cell_table_size_normalized)
    combined_cell_table_arcsinh_transformed = pd.concat(cell_table_arcsinh_transformed)

    ## qc expression table
    combined_cell_table_size_normalized_qc = pd.concat(cell_table_size_normalized_qc)
    combined_cell_table_arcsinh_transformed_qc = pd.concat(cell_table_arcsinh_transformed_qc)
    
    if (combined_cell_table_size_normalized.isnull().values.any() or combined_cell_table_arcsinh_transformed.isnull().values.any()):
        if ignore_na_warning == False:
            warnings.warn("Combining expression table introduced NA indicating that some marker are not present in all images or that markers are not consistently named across images. To proceed anyway set `ignore_na_warning=True` and rerun this step. Please, note that this most likely will lead to issue downstream. The better solution is to harmonise the marker names (using the previous step) to resolve this issuee.");
            continue_to_save = False
        elif ignore_na_warning == True and remove_non_shared_markers == False:
            warnings.warn("Combining expression table introduced NA and you have set `ignore_na_warning=True` to proceed anyway. Please, note that this most likely will lead to issue downstream.");  
            continue_to_save = True
        elif ignore_na_warning == True and remove_non_shared_markers == True:
            warnings.warn("Combining expression table introduced NA and you have set `remove_non_shared_markers=True` to proceed while removing the markers not present in all images.");  
            combined_cell_table_size_normalized.dropna(axis='columns', inplace=True)
            combined_cell_table_arcsinh_transformed.dropna(axis='columns', inplace=True)
            combined_cell_table_size_normalized_qc.dropna(axis='columns', inplace=True)
            combined_cell_table_arcsinh_transformed_qc.dropna(axis='columns', inplace=True)
            continue_to_save = True
    else:
        continue_to_save = True
        
        
    if continue_to_save:
        ## save combined feature matrices to the hard drive
        combined_cell_table_size_normalized_qc.to_csv(os.path.join(cell_table_dir_qc, 'cell_table_size_normalized.csv'),
                                        compression=compression, index=False)
        combined_cell_table_arcsinh_transformed_qc.to_csv(os.path.join(cell_table_dir_qc, 'cell_table_arcsinh_transformed.csv'),
                                            compression=compression, index=False)

        combined_cell_table_size_normalized.to_csv(os.path.join(cell_table_dir_qc, 'cell_table_size_normalized.csv'),
                                        compression=compression, index=False)
        combined_cell_table_arcsinh_transformed_qc.to_csv(os.path.join(cell_table_dir_qc, 'cell_table_arcsinh_transformed.csv'),
                                            compression=compression, index=False)

### 6.2 Spooxs

In [ ]:
if spooxs_output:
    print("Work in progress.")

### 6.2 Others

In [ ]:
if other_output:
    ## Implement your own conversion code here